In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import layers, optimizers, Sequential, metrics
from matplotlib import pyplot as plt
import  io

(x, y), (x_test, y_test) = keras.datasets.fashion_mnist.load_data() # 返回numpy
print(f"x shape:{x.shape}, y shape:{y.shape}")

def preprocess(x, y):

    x = tf.cast(x, dtype=tf.float32) / 255.
    y = tf.cast(y, dtype=tf.int32)
    return x,y

batchsz = 128
db = tf.data.Dataset.from_tensor_slices((x,y))
db = db.map(preprocess).shuffle(10000).batch(batchsz)
db_iter = iter(db)
sample = next(db_iter)
print('batch:', sample[0].shape, sample[1].shape)

db_test = tf.data.Dataset.from_tensor_slices((x_test,y_test))
db_test = db_test.map(preprocess).batch(batchsz)

x shape:(60000, 28, 28), y shape:(60000,)
batch: (128, 28, 28) (128,)


In [3]:
model = Sequential([
    layers.Dense(256, activation=tf.nn.relu), # [b, 784] => [b, 256]
    layers.Dense(128, activation=tf.nn.relu), # [b, 256] => [b, 128]
    layers.Dense(64, activation=tf.nn.relu), # [b, 128] => [b, 64]
    layers.Dense(32, activation=tf.nn.relu), # [b, 64] => [b, 32]
    layers.Dense(10) # [b, 32] => [b, 10], 330 = 32*10 + 10
])
model.build(input_shape=[None, 28*28])
model.summary()
# w = w - lr*grad
optimizer = optimizers.Adam(learning_rate=1e-3)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               200960    
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dense_4 (Dense)             (None, 10)                330       
                                                                 
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________


In [4]:
# 1.创建一个logs目录
# 2.启动tensorboard服务：tensorboard --logdir logs
# 3.网页查看localhost:6006
# 4.构建summary
import datetime
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = 'logs/' + current_time
summary_writer = tf.summary.create_file_writer(log_dir) 

In [7]:
# 准备绘图函数
def plot_to_image(figure):
  """Converts the matplotlib plot specified by 'figure' to a PNG image and
  returns it. The supplied figure is closed and inaccessible after this call."""
  # Save the plot to a PNG in memory.
  buf = io.BytesIO()
  plt.savefig(buf, format='png')
  # Closing the figure prevents it from being displayed directly inside
  # the notebook.
  plt.close(figure)
  buf.seek(0)
  # Convert PNG buffer to TF image
  image = tf.image.decode_png(buf.getvalue(), channels=4)
  # Add the batch dimension
  image = tf.expand_dims(image, 0)
  return image

def image_grid(images):
  """Return a 5x5 grid of the MNIST images as a matplotlib figure."""
  # Create a figure to contain the plot.
  figure = plt.figure(figsize=(10,10))
  for i in range(min(25, len(images))):# 每25张图片一组
    # Start next subplot.
    plt.subplot(5, 5, i + 1, title='name')
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(images[i], cmap=plt.cm.binary)
  
  return figure

In [8]:
for epoch in range(30):


    for step, (x,y) in enumerate(db):

        # x: [b, 28, 28] => [b, 784]
        # y: [b]
        x = tf.reshape(x, [-1, 28*28])

        with tf.GradientTape() as tape:
            # [b, 784] => [b, 10]
            logits = model(x)
            y_onehot = tf.one_hot(y, depth=10)
            # [b]
            loss_mse = tf.reduce_mean(tf.losses.MSE(y_onehot, logits))
            loss_ce = tf.losses.categorical_crossentropy(y_onehot, logits, from_logits=True)
            loss_ce = tf.reduce_mean(loss_ce)

        grads = tape.gradient(loss_ce, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))


        if step % 100 == 0:
            print(epoch, step, 'loss:', float(loss_ce), float(loss_mse))
            # 5.写入summary
            with summary_writer.as_default(): 
                tf.summary.scalar('train-loss', float(loss_ce), step=step) 


    # test
    total_correct = 0
    total_num = 0
    for x,y in db_test:

        # x: [b, 28, 28] => [b, 784]
        # y: [b]
        x = tf.reshape(x, [-1, 28*28])
        # [b, 10]
        logits = model(x)
        # logits => prob, [b, 10]
        prob = tf.nn.softmax(logits, axis=1)
        # [b, 10] => [b], int64
        pred = tf.argmax(prob, axis=1)
        pred = tf.cast(pred, dtype=tf.int32)
        # pred:[b]
        # y: [b]
        # correct: [b], True: equal, False: not equal
        correct = tf.equal(pred, y)
        correct = tf.reduce_sum(tf.cast(correct, dtype=tf.int32))

        total_correct += int(correct)
        total_num += x.shape[0]

    acc = total_correct / total_num
    print(epoch, 'test acc:', acc)

    # 5.写入summary
    val_images = x[:25] # 每25张图片一组
    val_images = tf.reshape(val_images, [-1, 28, 28, 1])
    with summary_writer.as_default():
        tf.summary.scalar('test-acc', float(total_correct/total_num), step=step)
        tf.summary.image("val-onebyone-images:", val_images, max_outputs=25, step=step)
        
        val_images = tf.reshape(val_images, [-1, 28, 28])
        figure  = image_grid(val_images)
        tf.summary.image('val-images:', plot_to_image(figure), step=step)

0 0 loss: 0.38710206747055054 18.419403076171875
0 100 loss: 0.3311314582824707 19.710655212402344
0 200 loss: 0.39904487133026123 18.829082489013672
0 300 loss: 0.43409693241119385 19.332101821899414
0 400 loss: 0.4294698238372803 17.10940933227539
0 test acc: 0.8643
1 0 loss: 0.16124220192432404 18.43413543701172
1 100 loss: 0.3001267611980438 23.568584442138672
1 200 loss: 0.22718653082847595 21.661556243896484
1 300 loss: 0.2219761461019516 29.63250160217285
1 400 loss: 0.34581488370895386 23.02088165283203
1 test acc: 0.8698
2 0 loss: 0.3177177309989929 23.92734718322754
2 100 loss: 0.2695233225822449 27.289321899414062
2 200 loss: 0.26662853360176086 22.334402084350586
2 300 loss: 0.2795843183994293 24.796428680419922
2 400 loss: 0.2523311376571655 21.879959106445312
2 test acc: 0.8715
3 0 loss: 0.19346833229064941 24.951229095458984
3 100 loss: 0.2666340470314026 32.21454620361328
3 200 loss: 0.2490980625152588 32.00535583496094
3 300 loss: 0.25985270738601685 26.90170669555664
